Analysing Corona-out-2 file

In [3]:
import json
file_1 = "../data/corona-out-2"
lines_file_1 = 0
tweets_file_1 = 0
retweets_file_1 = 0
field_counts_file_1 = {}
url_counts_file_1 = 0
sample_tweet_file_1 = None
sample_retweet_file_1 = None

with open(file_1, "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            lines_file_1 += 1
            is_retweet = data['text'].startswith('RT')
            retweets_file_1 += is_retweet
            tweets_file_1 += not is_retweet
            if is_retweet and not sample_retweet_file_1:
                sample_retweet_file_1 = data
            elif not is_retweet and not sample_tweet_file_1:
                sample_tweet_file_1 = data
            for key in data.keys():
                field_counts_file_1[key] = field_counts_file_1.get(key, 0) + 1
            url_counts_file_1 += len(data.get('entities', {}).get('urls', []))
        except json.JSONDecodeError:
            continue

print(f'Summary Report for {file_1}:')
print('Number of lines:', lines_file_1)
print('Number of tweets:', tweets_file_1)
print('Number of retweets:', retweets_file_1)
print('Number of URLs:', url_counts_file_1)
print('\nField Counts:')
for field, count in field_counts_file_1.items():
    print(field, ':', count)

print('\nSample Tweet from file 1:')
print(json.dumps(sample_tweet_file_1, indent=4))
print('\nSample Retweet from file 1:')
print(json.dumps(sample_retweet_file_1, indent=4))


Summary Report for ../data/corona-out-2:
Number of lines: 18518
Number of tweets: 7356
Number of retweets: 11162
Number of URLs: 4441

Field Counts:
created_at : 18518
id : 18518
id_str : 18518
text : 18518
source : 18518
truncated : 18518
in_reply_to_status_id : 18518
in_reply_to_status_id_str : 18518
in_reply_to_user_id : 18518
in_reply_to_user_id_str : 18518
in_reply_to_screen_name : 18518
user : 18518
geo : 18518
coordinates : 18518
place : 18518
contributors : 18518
retweeted_status : 11159
quoted_status_id : 3013
quoted_status_id_str : 3013
quoted_status : 2994
quoted_status_permalink : 2994
is_quote_status : 18518
quote_count : 18518
reply_count : 18518
retweet_count : 18518
favorite_count : 18518
entities : 18518
favorited : 18518
retweeted : 18518
filter_level : 18518
lang : 18518
timestamp_ms : 18518
display_text_range : 3183
extended_tweet : 2912
possibly_sensitive : 3449
extended_entities : 1084
withheld_in_countries : 81

Sample Tweet from file 1:
{
    "created_at": "Sun 

In [4]:
def check_for_duplicate_ids(filename):
    ids_seen = set()
    duplicate_ids = set()

    with open(filename, "r") as file:
        for line in file:
            try:
                tweet = json.loads(line)
                tweet_id = tweet.get('id')
                if tweet_id in ids_seen:
                    duplicate_ids.add(tweet_id)
                else:
                    ids_seen.add(tweet_id)
            except json.JSONDecodeError:
                continue  # Skipping lines that are not valid JSON

    return duplicate_ids

duplicates = check_for_duplicate_ids(file_1)

if duplicates:
    print(f"Found duplicates for the following tweet IDs: {duplicates}")
else:
    print("No duplicate tweet IDs found.")


Found duplicates for the following tweet IDs: {1249406404013764608, 1249407126847524864, 1249406332349882370, 1249407196334559235, 1249407921412333568, 1249405427361763333, 1249407955298115587, 1249404402139570183, 1249408006544134145, 1249408330784739328, 1249408430382755840, 1249408655671386115}


In [5]:
def find_first_duplicate_for_id(filename, tweet_id_to_find):
    ids_seen = {}
    first_duplicate_line = None

    with open(filename, "r") as file:
        for line_number, line in enumerate(file, 1):
            try:
                tweet = json.loads(line)
                tweet_id = tweet.get('id')
                if tweet_id == tweet_id_to_find:
                    if tweet_id in ids_seen and first_duplicate_line is None:
                        # Store the duplicate line and stop searching
                        first_duplicate_line = (line_number, line)
                        break
                    else:
                        ids_seen[tweet_id] = (line_number, line)
            except json.JSONDecodeError:
                continue 

    return ids_seen[tweet_id_to_find], first_duplicate_line


tweet_id_to_find = 1249406404013764608  # Replace with the ID you want to check

original, duplicate = find_first_duplicate_for_id(file_1, tweet_id_to_find)

if duplicate:
    print(f"Original occurrence - Line {original[0]}: {json.dumps(json.loads(original[1]), indent=4)}")
    print(f"First duplicate - Line {duplicate[0]}: {json.dumps(json.loads(duplicate[1]), indent=4)}")
else:
    print(f"No duplicate found for Tweet ID: {tweet_id_to_find}")


Original occurrence - Line 19185: {
    "created_at": "Sun Apr 12 18:37:54 +0000 2020",
    "id": 1249406404013764608,
    "id_str": "1249406404013764608",
    "text": "Corona",
    "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 1248439943384440839,
        "id_str": "1248439943384440839",
        "name": "Wolf 3.0 \u26f3\ufe0f",
        "screen_name": "Asap_infinityy",
        "location": "910",
        "url": "http://Golfwang.com",
        "description": "Back on my bs // 21 // rip @Asap_infinity",
        "translator_type": "none",
        "protected": false,
        "verified": false,
        "followers_count": 89,
        "friends_count": 171,
        "listed_count": 0,
        "favourites_count": 14

In [6]:
def find_tweet_with_id(file_name, tweet_id):
    with open(file_name, 'r') as file:
        for line in file:
            try:
                data = json.loads(line)
                if data.get("id") == tweet_id:
                    return data
            except json.JSONDecodeError:
                continue  # Skip invalid lines
    return None

# Tweet ID to find
target_tweet_id = 1249403770833981440

# Find and print the tweet
tweet = find_tweet_with_id(file_1, target_tweet_id)
if tweet:
    print(json.dumps(tweet, indent=4))
else:
    print(f"No tweet found with ID {target_tweet_id}.")

{
    "created_at": "Sun Apr 12 18:27:26 +0000 2020",
    "id": 1249403770833981440,
    "id_str": "1249403770833981440",
    "text": "RT @biannagolodryga: This image.\nThis quote.\n\n\u201cOne of the reasons the pandemic is hitting the country so hard is that no one appears to be\u2026",
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "truncated": false,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "in_reply_to_screen_name": null,
    "user": {
        "id": 270088915,
        "id_str": "270088915",
        "name": "Charles Mire",
        "screen_name": "charlesmire",
        "location": "Kitchener-Waterloo, Canada",
        "url": "http://www.structur3d.io",
        "description": "Avid outdoorsman, scientist, math nut....and I like beer. Co-founder of Structur3D Printing, tech for rubber additive manufacturing",
        

Analysing Corona-out-3 file

In [7]:
file_2 = "../data/corona-out-3"
lines_file_2 = 0
tweets_file_2 = 0
retweets_file_2 = 0
field_counts_file_2 = {}
url_counts_file_2 = 0
sample_tweet_file_2 = None
sample_retweet_file_2 = None

with open(file_2, "r") as f:
    for line in f:
        try:
            data = json.loads(line)
            lines_file_2 += 1
            is_retweet = data['text'].startswith('RT')
            retweets_file_2 += is_retweet
            tweets_file_2 += not is_retweet
            if is_retweet and not sample_retweet_file_2:
                sample_retweet_file_2 = data
            elif not is_retweet and not sample_tweet_file_2:
                sample_tweet_file_2 = data
            for key in data.keys():
                field_counts_file_2[key] = field_counts_file_2.get(key, 0) + 1
            url_counts_file_2 += len(data.get('entities', {}).get('urls', []))
        except json.JSONDecodeError:
            continue

print(f'Summary Report for {file_2}:')
print('Number of lines:', lines_file_2)
print('Number of tweets:', tweets_file_2)
print('Number of retweets:', retweets_file_2)
print('Number of URLs:', url_counts_file_2)
print('\nField Counts:')
for field, count in field_counts_file_2.items():
    print(field, ':', count)

print('\nSample Tweet from file 2:')
print(json.dumps(sample_tweet_file_2, indent=4))
print('\nSample Retweet from file 2:')
print(json.dumps(sample_retweet_file_2, indent=4))


Summary Report for ../data/corona-out-3:
Number of lines: 101916
Number of tweets: 40804
Number of retweets: 61112
Number of URLs: 32788

Field Counts:
created_at : 101916
id : 101916
id_str : 101916
text : 101916
display_text_range : 18124
source : 101916
truncated : 101916
in_reply_to_status_id : 101916
in_reply_to_status_id_str : 101916
in_reply_to_user_id : 101916
in_reply_to_user_id_str : 101916
in_reply_to_screen_name : 101916
user : 101916
geo : 101916
coordinates : 101916
place : 101916
contributors : 101916
is_quote_status : 101916
quote_count : 101916
reply_count : 101916
retweet_count : 101916
favorite_count : 101916
entities : 101916
extended_entities : 7267
favorited : 101916
retweeted : 101916
possibly_sensitive : 27433
filter_level : 101916
lang : 101916
timestamp_ms : 101916
retweeted_status : 61101
quoted_status_id : 16129
quoted_status_id_str : 16129
quoted_status : 16123
quoted_status_permalink : 16123
extended_tweet : 17760
withheld_in_countries : 46

Sample Tweet f

In [8]:
duplicates = check_for_duplicate_ids(file_2)

if duplicates:
    print(f"Found duplicates for the following tweet IDs: {duplicates}")
else:
    print("No duplicate tweet IDs found.")


Found duplicates for the following tweet IDs: {1254023086535573504, 1254022835435196417, 1254023839706574848, 1254030389192151041, 1254030135457652736, 1254032422926245888, 1254033154689699842, 1254034659484237824, 1254034661417705472, 1254035939116093441, 1254041962597855233, 1254047745435934722, 1254049508675158021, 1254052393664249857, 1254052800348270599, 1254053307032768517, 1254048755235450880, 1254042462655520768, 1254047845151318016, 1254048253961740288, 1254053555348135936, 1254043217475108864}


In [9]:
tweet_id_to_find = 1254059748556771329  # Replace with the ID you want to check

original, duplicate = find_first_duplicate_for_id(file_2, tweet_id_to_find)

if duplicate:
    print(f"Original occurrence - Line {original[0]}: {json.dumps(json.loads(original[1]), indent=4)}")
    print(f"First duplicate - Line {duplicate[0]}: {json.dumps(json.loads(duplicate[1]), indent=4)}")
else:
    print(f"No duplicate found for Tweet ID: {tweet_id_to_find}")

No duplicate found for Tweet ID: 1254059748556771329


Some Stats about both files.

In [10]:
total_lines = lines_file_1 + lines_file_2
total_tweets = tweets_file_1 + tweets_file_2
total_retweets = retweets_file_1 + retweets_file_2
total_urls = url_counts_file_1 + url_counts_file_2
combined_field_counts = field_counts_file_1

for key, value in field_counts_file_2.items():
    combined_field_counts[key] = combined_field_counts.get(key, 0) + value

print('Total Summary Report:')
print('Number of lines:', total_lines)
print('Number of tweets:', total_tweets)
print('Number of retweets:', total_retweets)
print('Total number of URLs:', total_urls)
print('\nCombined Field Counts:')
for field, count in combined_field_counts.items():
    print(field, ':', count)

Total Summary Report:
Number of lines: 120434
Number of tweets: 48160
Number of retweets: 72274
Total number of URLs: 37229

Combined Field Counts:
created_at : 120434
id : 120434
id_str : 120434
text : 120434
source : 120434
truncated : 120434
in_reply_to_status_id : 120434
in_reply_to_status_id_str : 120434
in_reply_to_user_id : 120434
in_reply_to_user_id_str : 120434
in_reply_to_screen_name : 120434
user : 120434
geo : 120434
coordinates : 120434
place : 120434
contributors : 120434
retweeted_status : 72260
quoted_status_id : 19142
quoted_status_id_str : 19142
quoted_status : 19117
quoted_status_permalink : 19117
is_quote_status : 120434
quote_count : 120434
reply_count : 120434
retweet_count : 120434
favorite_count : 120434
entities : 120434
favorited : 120434
retweeted : 120434
filter_level : 120434
lang : 120434
timestamp_ms : 120434
display_text_range : 21307
extended_tweet : 20672
possibly_sensitive : 30882
extended_entities : 8351
withheld_in_countries : 127
